# Question: Does the company promote diversity and inclusion?

Although many ESG issues may not be financially material to all industries, we believe that diversity and inclusion of all people -- regardless of gender, race, sexual orientation, religion, background, and other similar traits -- is one that can be viewed as material across the board.

Embracing different types of people is the moral thing to do, but it's also the best financial thing to do. In 2018, management consulting firm McKinsey & Co. released the report "Why Diversity Matters"; it examined data sets from 366 public companies in the U.S., Canada, Latin America, and the U.K. Companies in the top quartile for gender, racial, or ethnic diversity were more likely to generate financial returns above the national medians for their industry; the converse was also true. Meanwhile, McKinsey concluded that "diversity is probably a competitive differentiator that shifts market share toward more diverse companies over time."

Why? Plenty of data shows that diverse groups make better decisions than homogeneous ones. Narrow-minded in-group thinking, in which members are more likely to have similar experiences and viewpoints and come to the same conclusions, can be terrible for decision-making. Meanwhile, groups in which many different life experiences and perspectives are represented help set the stage for better decisions.

Underlining this principle is the correlation between what a company does and who runs it. For example, if a retailer sells jewelry aimed at female consumers, having an entirely male management team and board of directors would seem absurdly mismatched -- it would result in missing valuable perspectives about the very people it's aiming to attract as customers.

In ESG investing, we seek positive elements like a transparent workforce-diversity disclosure (some companies don't disclose their staff makeup at all); composition of its board of directors and executive leadership; and internal programs and policies that foster or support diversity -- adequate paid family leave, professional programs for people of color, and fair hiring practices.

Companies with poor diversity statistics, or lawsuits related to discrimination and harassment, will be noted for their heightened risk and probably disqualified from inclusion.

# Data Sources

- SEC Reports 

# Imports

In [2]:
# All necessary imports
import sec_edgar_downloader
from sec_edgar_downloader import Downloader
import re
import os
import shutil
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle

# Getting Companies

In [29]:
# Load in pickled list of companies and corresponding tickers
with open('./generated_data/companies_n_tickers.pickle','rb') as f:
    companies_n_tickers = pickle.load(f)

# Getting Data

In [30]:
def get_dandi_df(companies):
    
    dandi_df = pd.DataFrame(columns=['company','diversity_inclusion_mentions'])
    
    for a,b in companies:
        try:
            d = Downloader()
            d.get('DEF 14A', 
                  '{}'.format(b), 
                  after='2015-01-01',
                  download_details=False,
                  query = 'diversity inclusion')

            text_files = []
            for root, dirs, files in os.walk("/Users/MichaelWirtz/Desktop/portfolio/programming_projects/long_term_trading_bot/data_collection/sec-edgar-filings/{}".format(b)):
                for file in files:
                    if file.endswith(".txt"):
                         text_files.append(os.path.join(root, file))

            dandi_count = 0

            for file in text_files:
                content = open(file).read()
                valids = re.sub(r"[^a-z]+", ' ', content)
                content_count = valids.count('diversity') + valids.count('inclusion')
                dandi_count += content_count

            dandi_df = dandi_df.append({'company': a,
                                        'diversity_inclusion_mentions': dandi_count}, ignore_index=True)

            shutil.rmtree('./sec-edgar-filings')
        except:
            dandi_df = dandi_df.append({'company': a,
                                        'diversity_inclusion_mentions': 0}, ignore_index=True)
    
    dandi_df = dandi_df.sort_values(by='diversity_inclusion_mentions', ascending=False)
    
    return dandi_df  

In [31]:
# Define diversity_and_inclusion with function
diversity_and_inclusion = get_dandi_df(companies_n_tickers)

In [32]:
# Checking diversity_and_inclusion
diversity_and_inclusion

,company,diversity_inclusion_mentions
136,Deere & Co.,5055
377,Procter & Gamble,1318
358,Parker-Hannifin,1120
11,Agilent Technologies,792
368,Philip Morris International,507
...,...,...
469,ViacomCBS,0
470,ViacomCBS,0
471,Viatris,0
472,Viatris,0


In [33]:
# Pickling diversity and inclusion data frame
diversity_and_inclusion.to_pickle('./generated_data/diversity_n_inclusion.pickle')